In [5]:
!pip install requests
!pip install bs4
!pip install pandas

In [28]:
!pip install lxml

In [1]:
!pip install pandas openpyxl

In [86]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
from lxml import html

# Initialize lists to store the data
all_apartment_links = []
all_address = []
all_price = []
all_property_types = []
all_added_date = []
all_bedrooms = []
all_bathrooms = []
all_size = []
all_leasehold_freehold = []
all_council_tax = []
all_parking = []
all_garden = []
all_stations = []
all_agency_details = []

# Define the list of borough codes (hackney, tower hamlet)
boroughs = ["REGION%5E93953", "REGION%5E61417"]

# Set up the Chrome WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Loop through each borough
for borough in boroughs:
    # Initialize the starting index for each borough
    index = 0

    # Get the total number of listings for the current borough
    rightmove_initial = f"https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier={borough}&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false"
    driver.get(rightmove_initial)
    time.sleep(3)  # Allow time for the page to load
    number_of_listings = driver.find_element(By.CLASS_NAME, "searchHeader-resultCount").text
    number_of_listings = int(number_of_listings.replace(",", ""))

    # Loop through pages for the current borough
    while index < number_of_listings:
        try:
            # Construct the URL for the current page
            rightmove = f"https://www.rightmove.co.uk/property-for-sale/find.html?searchType=SALE&locationIdentifier={borough}&insId=1&index={index}&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&_includeSSTC=on&sortByPriceDescending=&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&newHome=&auction=false"
            driver.get(rightmove)
            time.sleep(3)  # Allow time for the page to load

            # Get the list of apartments
            apartments = driver.find_elements(By.CLASS_NAME, "l-searchResult")

            # Check if there are no more apartments found
            if not apartments:
                print(f"No more apartments found in borough {borough}.")
                break

            # Clear the lists for the current page to avoid duplicates
            page_apartment_links = []
            page_address = []
            page_price = []
            page_added_date = []

            for apartment in apartments:
                # Append link
                try:
                    link = apartment.find_element(By.CLASS_NAME, "propertyCard-link").get_attribute("href")
                    page_apartment_links.append(link)
                except:
                    page_apartment_links.append("N/A")

                # Append address
                try:
                    address = apartment.find_element(By.CLASS_NAME, "propertyCard-address").text.strip()
                    page_address.append(address)
                except:
                    page_address.append("N/A")

                # Append price
                try:
                    price = apartment.find_element(By.CLASS_NAME, "propertyCard-priceValue").text.strip()
                    page_price.append(price)
                except:
                    page_price.append("N/A")

                # Append details
                try:
                    details = apartment.find_element(By.CLASS_NAME, "propertyCard-branchSummary-addedOrReduced").text.strip()
                    page_added_date.append(details)
                except:
                    page_added_date.append("N/A")

            # Visit each property link to extract more details
            for link in page_apartment_links:
                if link == "N/A":
                    all_property_types.append("N/A")
                    all_bedrooms.append("N/A")
                    all_bathrooms.append("N/A")
                    all_size.append("N/A")
                    all_leasehold_freehold.append("N/A")
                    all_council_tax.append("N/A")
                    all_parking.append("N/A")
                    all_garden.append("N/A")
                    all_stations.append("N/A")
                    all_agency_details.append("N/A")
                    continue

                try:
                    driver.get(link)
                    time.sleep(3)  # Allow time for the page to load

                    # Get the page source and parse it with lxml
                    page_source = driver.page_source
                    tree = html.fromstring(page_source)

                    try:
                        property_type = tree.xpath('//*[@id="info-reel"]/div[1]/dd/span/p/text()')[0]
                    except:
                        property_type = "N/A"
                    all_property_types.append(property_type)

                    try:
                        bedrooms = tree.xpath('//*[@id="info-reel"]/div[2]/dd/span/p/text()')[0]
                    except:
                        bedrooms = "N/A"
                    all_bedrooms.append(bedrooms)

                    try:
                        bathrooms = tree.xpath('//*[@id="info-reel"]/div[3]/dd/span/p/text()')[0]
                    except:
                        bathrooms = "N/A"
                    all_bathrooms.append(bathrooms)

                    try:
                        size1 = tree.xpath('//*[@id="info-reel"]/div[4]/dd/span/p[1]/text()')
                        size2 = tree.xpath('//*[@id="info-reel"]/div[4]/dd/span/p[2]/text()')
                        size = ", ".join(filter(None, [size1[0] if size1 else None, size2[0] if size2 else None]))
                        if not size:
                            size = "N/A"
                    except:
                        size = "N/A"
                    all_size.append(size)

                    # Use driver.find_element for leasehold_freehold
                    try:
                        leasehold_freehold = driver.find_element(By.XPATH, '//*[@id="info-reel"]/div[5]').text
                    except:
                        leasehold_freehold = "N/A"
                    all_leasehold_freehold.append(leasehold_freehold)

                    try:
                        council_tax = tree.xpath("//dd[@class='_2zXKe70Gdypr_v9MUDoVCm']/text()")[0]
                    except:
                        council_tax = "N/A"
                    all_council_tax.append(council_tax)

                    try:
                        parking = tree.xpath("(//span[@class='_3rQAUgsu_ICdA55QUiiUxg'])[1]/text()")[0]
                    except:
                        parking = "N/A"
                    all_parking.append(parking)

                    try:
                        garden = tree.xpath("(//span[@class='_3rQAUgsu_ICdA55QUiiUxg'])[2]/text()")[0]
                    except:
                        garden = "N/A"
                    all_garden.append(garden)

                    try:
                        station_elements = tree.xpath("//div[@class='mlEuHXZpfrrzJtwlRmwBe']")
                        stations = ", ".join([station.text_content() for station in station_elements])
                    except:
                        stations = "N/A"
                    all_stations.append(stations)

                    # Append agency details using driver.find_element
                    try:
                        agency_details = driver.find_element(By.XPATH, '//*[@id="root"]/main/div/div[2]/div/article[4]/div[1]').text
                    except:
                        agency_details = "N/A"
                    all_agency_details.append(agency_details)

                except Exception as err:
                    print(f"Error occurred while fetching details for link {link}: {err}")
                    all_property_types.append("N/A")
                    all_bedrooms.append("N/A")
                    all_bathrooms.append("N/A")
                    all_size.append("N/A")
                    all_leasehold_freehold.append("N/A")
                    all_council_tax.append("N/A")
                    all_parking.append("N/A")
                    all_garden.append("N/A")
                    all_stations.append("N/A")
                    all_agency_details.append("N/A")

            # Add current page data to the main lists
            all_apartment_links.extend(page_apartment_links)
            all_address.extend(page_address)
            all_price.extend(page_price)
            all_added_date.extend(page_added_date)

            # Increment the index for pagination
            index += 24  # Assuming each page shows 24 listings

            # Add a delay to avoid being blocked by the website
            time.sleep(2)

        except Exception as err:
            print(f"Error occurred: {err}")
            break

# Ensure all lists have the same length
min_length = min(len(all_apartment_links), len(all_address), len(all_price), len(all_property_types), len(all_added_date), len(all_bedrooms), len(all_bathrooms), len(all_size), len(all_leasehold_freehold), len(all_council_tax), len(all_parking), len(all_garden), len(all_stations), len(all_agency_details))
all_apartment_links = all_apartment_links[:min_length]
all_address = all_address[:min_length]
all_price = all_price[:min_length]
all_property_types = all_property_types[:min_length]
all_added_date = all_added_date[:min_length]
all_bedrooms = all_bedrooms[:min_length]
all_bathrooms = all_bathrooms[:min_length]
all_size = all_size[:min_length]
all_leasehold_freehold = all_leasehold_freehold[:min_length]
all_council_tax = all_council_tax[:min_length]
all_parking = all_parking[:min_length]
all_garden = all_garden[:min_length]
all_stations = all_stations[:min_length]
all_agency_details = all_agency_details[:min_length]

# Convert data to DataFrame
data = {
    "Links": all_apartment_links,
    "Address": all_address,
    "Price": all_price,
    "Property Type": all_property_types,
    "Added Date": all_added_date,
    "Bedrooms": all_bedrooms,
    "Bathrooms": all_bathrooms,
    "Size": all_size,
    "Leasehold/Freehold": all_leasehold_freehold,
    "Council Tax": all_council_tax,
    "Parking": all_parking,
    "Garden": all_garden,
    "Stations": all_stations,
    "Agency Details": all_agency_details
}
df = pd.DataFrame.from_dict(data)

# Save DataFrame to CSV file
df.to_csv("all_listings.csv", encoding="utf-8", header=True, index=False)

# Display the results
print("Total listings scraped:", len(all_apartment_links))
print("Sample data:")
print(df.head())

# Close the WebDriver
driver.quit()


No more apartments found in borough REGION%5E93953.
No more apartments found in borough REGION%5E61417.
Total listings scraped: 2100
Sample data:
                                               Links  \
0  https://www.rightmove.co.uk/properties/1472160...   
1  https://www.rightmove.co.uk/properties/1489738...   
2  https://www.rightmove.co.uk/properties/1426485...   
3  https://www.rightmove.co.uk/properties/1469238...   
4  https://www.rightmove.co.uk/properties/1487114...   

                                    Address       Price Property Type  \
0           Kimpton Court, Murrain Road, N4    £500,000          Flat   
1                     Principal Place, EC2A  £9,900,000          Flat   
2             Principal Tower, London, EC2A  £9,900,000     Penthouse   
3  Principal Tower, 2 Principal Place, EC2A  £9,900,000          Flat   
4               Principal Tower, Shoreditch  £9,900,000     Penthouse   

            Added Date Bedrooms Bathrooms                   Size  \
0  Added o

In [81]:
import pandas as pd

# Read the CSV file
csv_file = "all_listings.csv"
df = pd.read_csv(csv_file)

# Save as an Excel file
excel_file = "all_listings.xlsx"
df.to_excel(excel_file, index=False)

print(f"File converted and saved as {excel_file}")


File converted and saved as all_listings.xlsx


In [79]:
# Clean the Price column
# Get the date of yesterday
# Clean the Added Date column
# Define the cleaning function
# Convert Size values to numeric (removing 'sq m' and handling commas)
# Clean the Bedrooms column to ensure it only contains numeric values and convert to integer
# Calculate the overall average size for one-bedroom properties
# Fill empty size cells with the average size multiplied by the number of bedrooms
# Convert the size back to the original format with one decimal place
# Clean the Leasehold/Freehold column
# Clean the Stations column to keep only the shortest distance
# Clean the Agency Details column to keep only the name of the agency
# Clean the Council Tax column
# Define the replacements for the Parking column
# Replace the values in the Parking column
# Clean the Garden column
# Remove rows where the Bathrooms column contains text
# Remove rows with empty cells in the 'Property Type' column and those with 'Plot' as the 'Property Type'
# Rename the column 
# Replace any remaining nan values with 0
# Create Unique_ID column by combining Price, Property Type, Address, Bedrooms, and Bathrooms
# Remove duplicate rows based on the Unique_ID column
# Extract postcode from Address column

import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np

# Read the CSV file
csv_file = "all_listings.csv"
df = pd.read_csv(csv_file)

# Clean the Price column
df['Price'] = df['Price'].str.replace('£', '').str.replace(',', '').astype(int)

# Get the date of yesterday
yesterday = (datetime.now() - timedelta(1)).strftime("%d/%m/%Y")

# Clean the Added Date column
def process_date(date_str):
    if isinstance(date_str, float) and pd.isna(date_str):
        return ""
    date_str = str(date_str)
    if "Added on" in date_str:
        return date_str.replace("Added on ", "")
    elif "Reduced yesterday" in date_str or "Added yesterday" in date_str:
        return yesterday
    elif "Reduced on" in date_str:
        return date_str.replace("Reduced on ", "")
    else:
        return date_str

df['Added Date'] = df['Added Date'].apply(process_date)

# Define the cleaning function
def clean_size(size_str):
    return ', '.join([s for s in str(size_str).split(', ') if 'sq m' in s])

# Apply the cleaning function
df['Size'] = df['Size'].apply(clean_size)

# Convert Size values to numeric (removing 'sq m' and handling commas)
def parse_size(size_str):
    try:
        return float(size_str.replace('sq m', '').replace(',', '').strip())
    except:
        return np.nan

df['Size'] = df['Size'].apply(parse_size)

# Clean the Bedrooms column to ensure it only contains numeric values and convert to integer
def clean_bedrooms(bedroom_str):
    try:
        return int(float(bedroom_str))
    except ValueError:
        return np.nan

df['Bedrooms'] = df['Bedrooms'].apply(clean_bedrooms)

# Calculate the overall average size for one-bedroom properties
average_size_one_bedroom = df['Size'].mean() / df['Bedrooms'].mean()

# Fill empty size cells with the average size multiplied by the number of bedrooms
def fill_empty_size(row):
    if pd.isna(row['Size']) or row['Size'] == '':
        try:
            return average_size_one_bedroom * int(row['Bedrooms'])
        except ValueError:
            return np.nan
    return row['Size']

df['Size'] = df.apply(fill_empty_size, axis=1)

# Convert the size back to the original format with one decimal place
df['Size'] = df['Size'].apply(lambda x: f"{x:.1f}" if not pd.isna(x) else '')

# Clean the Leasehold/Freehold column
def clean_leasehold_freehold(value):
    if pd.isna(value) or "Ask agent" in value:
        return "0"
    return re.sub(r'TENURE\s*', '', value).strip()

df['Leasehold/Freehold'] = df['Leasehold/Freehold'].astype(str).apply(clean_leasehold_freehold)

# Clean the Stations column to keep only the shortest distance
def get_shortest_station(stations_str):
    distances = re.findall(r'(\d+\.\d+)', stations_str)
    shortest_distance = min(distances, key=float) if distances else "0"
    return shortest_distance

df['Stations'] = df['Stations'].apply(get_shortest_station)

# Clean the Agency Details column to keep only the name of the agency
df['Agency Details'] = df['Agency Details'].apply(lambda x: str(x).split(',')[0])

# Clean the Council Tax column
def clean_council_tax(tax_str):
    if isinstance(tax_str, float) and pd.isna(tax_str):
        return "0"
    tax_str = str(tax_str)
    if "Ask agent" in tax_str or "Ask developer" in tax_str:
        return "0"
    tax_str = tax_str.replace("Band: ", "")
    if tax_str == "TBC":
        return "0"
    return tax_str

df['Council Tax'] = df['Council Tax'].apply(clean_council_tax)

# Define the replacements for the Parking column
parking_replacements = {
    'Ask agent': 'FALSE',
    'Private': 'TRUE',
    'Allocated': 'TRUE',
    'Off street': 'TRUE',
    'On street': 'TRUE',
    'Garage': 'TRUE',
    'Underground': 'TRUE',
    'Permit': 'TRUE',
    'Ask developer': 'FALSE',
    'Residents': 'TRUE',
    'Disabled parking': 'TRUE',
    'No parking': 'FALSE',
    'Gated': 'TRUE',
    'Yes' : 'TRUE',
    pd.NA: 'FALSE',
    'nan': 'FALSE'  
}

# Replace the values in the Parking column
df['Parking'] = df['Parking'].replace(parking_replacements)

# Clean the Garden column
def clean_garden(garden_str):
    if isinstance(garden_str, float) and pd.isna(garden_str):
        return 'FALSE'
    garden_str = str(garden_str)
    if "Ask agent" in garden_str or "Ask developer" in garden_str:
        return 'FALSE'
    return 'TRUE'

df['Garden'] = df['Garden'].apply(clean_garden)

# Remove rows where the Bathrooms column contains text
def is_valid_bathroom(bathroom_str):
    try:
        int(bathroom_str)
        return True
    except ValueError:
        return False

df = df[df['Bathrooms'].apply(is_valid_bathroom)]

# Remove rows with empty cells in the 'Property Type' column and those with 'Plot' as the 'Property Type'
df = df[df['Property Type'].notna() & (df['Property Type'] != '') & (df['Property Type'].str.lower() != 'plot')]

# Rename the column 
df.rename(columns={'Size': 'Size (sq m)'}, inplace=True)
df.rename(columns={'Stations': 'Stations (miles)'}, inplace=True)
df.rename(columns={'Price': 'Price (£)'}, inplace=True)
df.rename(columns={'Leasehold/Freehold': 'TENURE'}, inplace=True)

# Replace any remaining nan values with 0
df = df.replace({np.nan: '0', '': '0'})

# Create Unique_ID column by combining Price, Property Type, Address, Bedrooms, and Bathrooms
df['Unique_ID'] = df.apply(lambda row: f"{row['Price (£)']}_{row['Property Type']}_{row['Address']}_{row['Bedrooms']}_{row['Bathrooms']}", axis=1)

# Remove duplicate rows based on the Unique_ID column
df = df.drop_duplicates(subset=['Unique_ID'])

# Extract postcode from Address column
def extract_postcode(address):
    parts = address.split(',')
    last_part = parts[-1].strip()
    
    # Check if the last part has a number
    if any(char.isdigit() for char in last_part):
        # If it's a full postcode (contains a space), return the first part only
        if ' ' in last_part:
            first_part = last_part.split(' ')[0]
            # Check if the first part contains a number
            if any(char.isdigit() for char in first_part):
                return first_part
        else:
            # Return the last part as it is if it contains a number
            return last_part
    return '0'  # Return '0' if no postcode is found

df['Address'] = df['Address'].apply(extract_postcode)

# Convert the Bedrooms column to integer type
df['Bedrooms'] = df['Bedrooms'].astype(int)

# Drop the 'Links' column
df = df.drop(columns=['Links'])

# Save the cleaned data to a CSV file
cleaned_csv_file = "all_listings_cleaned.csv"
df.to_csv(cleaned_csv_file, index=False)

print(f"File cleaned and saved as {cleaned_csv_file}")


# Save as an Excel file
excel_file = "all_listings_cleaned.xlsx"
df.to_excel(excel_file, index=False)

print(f"File converted and saved as {excel_file}")


File cleaned and saved as all_listings_cleaned.csv
File converted and saved as all_listings_cleaned.xlsx
